In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from cqp.bitrate_model.trainer import *
from cqp.cqp_analytic_model_train import *
from cqp.bitrate_model.plot import *
from cqp.bitrate_model.util import *
from cqp.bitrate_model.glm import *

In [ ]:
BITRATE_MODEL = BitrateModel
INIT_MODEL_PARAMETER = ModelParameter()

In [ ]:
data_files = list(Path.cwd().joinpath("data").glob("*.csv"))

dfw = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TRAINING_SET))
dfw_test = DataFrameWrapper(read_df_by_keys(data_files, KEYS.TEST_SET))

In [ ]:
video_keys = dfw.video_keys
test_video_keys = dfw_test.video_keys

ax = plt.gca()
plot_sa_ta(ax, dfw.df, video_keys, "Training")
plot_sa_ta(ax, dfw_test.df, test_video_keys, "Validation")
plt.legend()

### Maximum Bitrate

$$R_{max,st} = \rho_0\cdot SA + \rho_1\cdot\frac{SA}{TA} + \rho_2\cdot SA\cdot TA + \rho_3$$

In [ ]:
df_r_max = dfw.include()

# Non linear least square fitting for Rmax
r_max_mae = run_loocv(df_r_max, r_max_loocv_handle, BITRATE_MODEL)
print(r_max_mae)

r_max_trainer = RMaxTrainer(df_r_max, BITRATE_MODEL, INIT_MODEL_PARAMETER)
rho = r_max_trainer.train()
expected_rmax, predicted_rmax = r_max_trainer.validate(rho)

plot_content_dependent_parameter(r_max_trainer.keys, expected_rmax, predicted_rmax, rho, "Rmax", "Rho")

### Spatial Correction Factor

$$SCF(q_p, q_{p,min}) = \left (\frac{q_p}{q_{p,min}}\right )^{-a}$$

$$a_{st} = \alpha_0\cdot SA + \alpha_1\cdot SA\cdot TA + \alpha_2$$

In [ ]:
df_qp = dfw.include(variable_qp=True)

plot_correction_factor_bitrate_contribution(df_qp, KEYS.QP, "QP")

# Non linear least square fitting for SCF
scf_trainer = SCFTrainer(df_qp, BITRATE_MODEL, INIT_MODEL_PARAMETER)
a = scf_trainer.train_cf()
expected_scf, predicted_scf = scf_trainer.validate_cf(a)

plot_correction_factor(scf_trainer.X, expected_scf, predicted_scf, a, "$q_p$", "SCF", "a", unique=True)

In [ ]:
alpha_mae = run_loocv(df_qp, a_st_loocv_handle, BITRATE_MODEL)
print(alpha_mae)

alpha = scf_trainer.train_st(a)
expected_a, predicted_a = scf_trainer.validate_st(a, alpha)

plot_content_dependent_parameter(scf_trainer.keys, expected_a, predicted_a, alpha, "a", "alpha")

In [ ]:
expected_scf, predicted_scf = scf_trainer.validate(alpha)

plot_correction_factor_st(scf_trainer.X, expected_scf, predicted_scf, "QP", "SCF", unique=True)

### Temporal Correction Factor

$$TCF(f, f_{max}) = \left (\frac{f}{f_{max}}\right )^{b}$$

$$b_{st} = 1$$

In [ ]:
df_rate = dfw.include(variable_rate=True)

plot_correction_factor_bitrate_contribution(df_rate, KEYS.RATE, "Rate")

# Non linear least square fitting for SCF
tcf_trainer = TCFTrainer(df_rate, BITRATE_MODEL, INIT_MODEL_PARAMETER)
b = tcf_trainer.train_cf()
expected_tcf, predicted_tcf = tcf_trainer.validate_cf(b)

plot_correction_factor(tcf_trainer.X, expected_tcf, predicted_tcf, b, "$f$", "TCF", "b", unique=True)

In [ ]:
beta_mae = run_loocv(df_rate, b_st_loocv_handle, BITRATE_MODEL)
print(beta_mae)

beta = tcf_trainer.train_st(b)
expected_b, predicted_b = tcf_trainer.validate_st(b, beta)

plot_content_dependent_parameter(tcf_trainer.keys, expected_b, predicted_b, beta, "b", "beta")

In [ ]:
expected_tcf, predicted_tcf = tcf_trainer.validate(beta)

plot_correction_factor_st(tcf_trainer.X, expected_tcf, predicted_tcf, "Rate", "TCF", unique=True)

### GOP Length Correction Factor

$$NCF(n) = c\cdot\left (\frac{1}{n}\right )^{d} + e$$

$$c_{st} = \gamma_0\cdot \log(TA) + \gamma_1\cdot TA + \gamma_2$$

$$d_{st} = \delta_0\cdot\frac{TA}{SA} + \delta_1$$

$$e_{st} = \epsilon_0\cdot \log(TA) + \epsilon_1\cdot TA + \epsilon_2$$

In [ ]:
df_gop = dfw.include(variable_gop=True)

plot_correction_factor_bitrate_contribution(df_gop, KEYS.GOP, "GOP")

# Non linear least square fitting for NCF
ncf_trainer = NCFTrainer(df_gop, BITRATE_MODEL, INIT_MODEL_PARAMETER)
c_d_e = ncf_trainer.train_cf()
expected_ncf, predicted_ncf = ncf_trainer.validate_cf(c_d_e)

plot_correction_factor(ncf_trainer.X, expected_ncf, predicted_ncf, c_d_e, "$n$", "NCF", "[c, d, e]", unique=True)

c, d, e = NCFTrainer.split_c_d_e(c_d_e)

In [ ]:
gamma_mae = run_loocv(df_gop, c_st_loocv_handle, BITRATE_MODEL)
print(gamma_mae)

gamma = ncf_trainer.train_c_st(c)
expected_c, predicted_c = ncf_trainer.validate_c_st(c, gamma)

plot_content_dependent_parameter(ncf_trainer.keys, expected_c, predicted_c, gamma, "c", "gamma")

In [ ]:
delta_mae = run_loocv(df_gop, d_st_loocv_handle, BITRATE_MODEL)
print(delta_mae)

delta = ncf_trainer.train_d_st(d)
expected_d, predicted_d = ncf_trainer.validate_d_st(d, delta)

plot_content_dependent_parameter(ncf_trainer.keys, expected_d, predicted_d, delta, "d", "delta")

In [ ]:
epsilon_mae = run_loocv(df_gop, e_st_loocv_handle, BITRATE_MODEL)
print(epsilon_mae)

epsilon = ncf_trainer.train_e_st(e)
expected_e, predicted_e = ncf_trainer.validate_e_st(e, epsilon)

plot_content_dependent_parameter(ncf_trainer.keys, expected_e, predicted_e, epsilon, "e", "epsilon")

In [ ]:
expected_ncf, predicted_ncf = ncf_trainer.validate((gamma, delta, epsilon))

plot_correction_factor_st(ncf_trainer.X, expected_ncf, predicted_ncf, "GOP len", "NCF", unique=True)

### Resolution Correction Factor

$$RCF(r, r_{max}) = \left (\frac{r}{r_{max}} \right )^j$$

$$j_{st} = \varphi_0\cdot SA\cdot TA + \varphi_1$$

In [ ]:
df_resolution = dfw.include(variable_res=True)
plot_correction_factor_bitrate_contribution(df_resolution, KEYS.RES, "Resolution")

# Non linear least square fitting for RCF
rcf_trainer = RCFTrainer(df_resolution, BITRATE_MODEL, INIT_MODEL_PARAMETER)
j = rcf_trainer.train_cf()
expected_rcf, predicted_rcf = rcf_trainer.validate_cf(j)

plot_correction_factor(rcf_trainer.X, expected_rcf, predicted_rcf, j, "$r$", "RCF", "j", unique=True)

In [ ]:
phi_mae = run_loocv(df_resolution, j_st_loocv_handle, BITRATE_MODEL)
print(phi_mae)

phi = rcf_trainer.train_st(j)
expected_j, predicted_j = rcf_trainer.validate_st(j, phi)

plot_content_dependent_parameter(rcf_trainer.keys, expected_j, predicted_j, phi, "j", "phi")

In [ ]:
expected_rcf, predicted_rcf = rcf_trainer.validate(phi)

plot_correction_factor_st(rcf_trainer.X, expected_rcf, predicted_rcf, "Width", "RCF", unique=True)

### Gaussian Kernel Correction Factor

$$GCF(k, k_{min}) = l\cdot\left (\frac{k_{min}}{k}\right )^{m} + o$$

$$l_{st} = \lambda_0\cdot SA + \lambda_1$$

In [ ]:
df_ksize = dfw.include(variable_k_size=True)
plot_correction_factor_bitrate_contribution(df_ksize, KEYS.KSIZE, "KSize")

# Non linear least square fitting for GCF
gcf_trainer = GCFTrainer(df_ksize, BITRATE_MODEL, INIT_MODEL_PARAMETER)
l_m_o = gcf_trainer.train_cf()
expected_gcf, predicted_gcf = gcf_trainer.validate_cf(l_m_o)

plot_correction_factor(gcf_trainer.X, expected_gcf, predicted_gcf, l_m_o, "$k$", "GCF", "[l, m, o]", unique=True)

l, m, o = GCFTrainer.split_l_m_o(l_m_o)

In [ ]:
lambda_mae = run_loocv(df_ksize, l_st_loocv_handle, BITRATE_MODEL)
print(lambda_mae)

lambda_ = gcf_trainer.train_l_st(l)
expected_l, predicted_l = gcf_trainer.validate_l_st(l, lambda_)

plot_content_dependent_parameter(gcf_trainer.keys, expected_l, predicted_l, lambda_, "l", "lambda")

In [ ]:
mu_mae = run_loocv(df_ksize, m_st_loocv_handle, BITRATE_MODEL)
print(mu_mae)

mu = gcf_trainer.train_m_st(m)
expected_m, predicted_m = gcf_trainer.validate_m_st(m, mu)

plot_content_dependent_parameter(gcf_trainer.keys, expected_m, predicted_m, mu, "m", "mu")

In [ ]:
omega_mae = run_loocv(df_ksize, o_st_loocv_handle, BITRATE_MODEL)
print(omega_mae)

omega = gcf_trainer.train_o_st(o)
expected_o, predicted_o = gcf_trainer.validate_o_st(o, omega)

plot_content_dependent_parameter(gcf_trainer.keys, expected_o, predicted_o, omega, "o", "omega")

In [ ]:
expected_gcf, predicted_gcf = gcf_trainer.validate((lambda_, mu, omega))

plot_correction_factor_st(gcf_trainer.X, expected_gcf, predicted_gcf, "KSize", "GCF", unique=True)

### Gaussian Standard Deviation Correction Factor

$$GSDCF(\sigma, \sigma_{min}) = t\cdot\left (\frac{\sigma_{min}}{\sigma}\right )^{u} + v$$

$$t_{st} = \tau_0\cdot\log (SA\cdot TA) + \tau_1$$

$$u_{st} = \upsilon_0\cdot SA\cdot TA + \upsilon_1$$

$$v_{st} = \nu_0\cdot\log (SA\cdot TA) + \nu_1$$

In [ ]:
df_gauss = dfw.include(variable_sigma=True, variable_k_size=True)
df_sigma = df_gauss.loc[df_gauss[KEYS.KSIZE] == 3]
plot_correction_factor_bitrate_contribution(df_sigma, KEYS.SIGMA, "sigma")

# Non linear least square fitting for GCF
sdcf_trainer = SDCFTrainer(df_sigma, BITRATE_MODEL, INIT_MODEL_PARAMETER)
t_u_v = sdcf_trainer.train_cf()
expected_sdcf, predicted_sdcf = sdcf_trainer.validate_cf(t_u_v)

plot_correction_factor(sdcf_trainer.X, expected_sdcf, predicted_sdcf, t_u_v, "$\sigma$", "SDCF", "[t, u, v]", unique=True)

t, u, v = SDCFTrainer.split_t_u_v(t_u_v)

In [ ]:
tau_mae = run_loocv(df_sigma, t_st_loocv_handle, BITRATE_MODEL)
print(tau_mae)

tau = sdcf_trainer.train_t_st(t)
expected_t, predicted_t = sdcf_trainer.validate_t_st(t, tau)

plot_content_dependent_parameter(sdcf_trainer.keys, expected_t, predicted_t, tau, "t", "tau")

In [ ]:
upsilon_mae = run_loocv(df_sigma, u_st_loocv_handle, BITRATE_MODEL)
print(upsilon_mae)

upsilon = sdcf_trainer.train_u_st(u)
expected_u, predicted_u = sdcf_trainer.validate_u_st(u, upsilon)

plot_content_dependent_parameter(sdcf_trainer.keys, expected_u, predicted_u, upsilon, "u", "upsilon")

In [ ]:
nu_mae = run_loocv(df_sigma, v_st_loocv_handle, BITRATE_MODEL)
print(nu_mae)

nu = sdcf_trainer.train_v_st(v)
expected_v, predicted_v = sdcf_trainer.validate_v_st(v, nu)

plot_content_dependent_parameter(sdcf_trainer.keys, expected_v, predicted_v, nu, "v", "nu")

In [ ]:
expected_sdcf, predicted_sdcf = sdcf_trainer.validate((tau, upsilon, nu))

plot_correction_factor_st(sdcf_trainer.X, expected_sdcf, predicted_sdcf, "$\sigma$", "SDCF", unique=True)

In [ ]:
# Evaluate and visualize bitrate
bitrate_model = BITRATE_MODEL(INIT_MODEL_PARAMETER)
for key in dfw_test.video_keys.tolist():
    bitrate_measured, bitrate_estimated = validate_bitrates_for_video(
        get_df_for_evaluation(dfw_test, BITRATE_MODEL, False), bitrate_model, key,
    )
    plot_bitrates(bitrate_measured, bitrate_estimated, key)
    if key == KEYS.SINGLE_VIDEO:
        save_plot(key)